In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 패키지 이용할 때
import sys, requests, bs4, time, urllib, csv, re, pickle # pickle은 리스트 가져오는 것
sys.path.append('/content/drive/MyDrive/additional_package')

from selenium import webdriver
from urllib.request import urlopen
import pandas as pd

In [ ]:
# Colab에서 Selenium 사용하기
!apt-get update
!apt install chromium-chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# 영화 제목 list 불러오기
with open("/content/drive/MyDrive/Quotes_and_Recommend/data/movie_title.pickle", "rb") as file:
  title_list = pickle.load(file)
file.close
print(title_list[0:5])
print(len(title_list))

['스펜서', '더 배트맨', '나이트메어 앨리', '리코리쉬 피자', '굿 보스']
1012


In [ ]:
# 영화 코드 리스트 가져오기
def get_code_list(movie):
  main_url = f"https://movie.naver.com/movie/search/result.naver?query={movie}&section=all&ie=utf8"
    # f 꼭 넣기. 안 넣으면 {movie}가 변수로 처리되지 않음
    # 코덱 안 맞아서 다른 코드 사용 
    ## html = urllib.request.urlopen(main_url)
  html = requests.get(main_url) # 호출
  soup_movie = bs4.BeautifulSoup(html.text, 'lxml') # html.parser보다 lxml이 성능 좋음
  href_list = []
  code_list = []
  try: 
    hrefs = soup_movie.find("ul",{"class":"search_list_1"}).find_all("li") # 영화 검색 목록 전체 가져오기 
    for href in hrefs: # 영화 검색창의 url: href.li.p.a["href"]
      get_code=re.split("=",  href.p.a["href"]) # url을 = 을 기준으로 나누기
      code_list.append(get_code[1]) # =를 기준으로 둘로 나뉜 것 중 code number만 가져오기
    return code_list
  except:
    return code_list

In [ ]:
# 명대사 수 찾기
def get_num(movie_code):
  #for movie_code in code_list:
  quotes_url = f"https://movie.naver.com/movie/bi/mi/scriptAndRelate.naver?code={movie_code}"
  driver.get(quotes_url)
  try:
    driver.switch_to.frame('ifr_lines') # iframe name
    #time.sleep(0.2)
    html_quotes = driver.page_source
    soup_quotes = bs4.BeautifulSoup(html_quotes, 'lxml')

    # 명대사가 여러 쪽이면
    num_quotes = soup_quotes.select_one("#iframeDiv > div.top_behavior.no_border > span").text # 총 명대사 수 가져오기
    num_str = re.findall("\d", num_quotes)
    num =int("".join(num_str)) # 문자열로 구해온 숫자 합쳐서 정수형으로 만들기
    return num

  except:
    return 0 
    # None값 나오면 다음 함수에서 처리 안 됨. 다음 함수에서 int 요구하니까 0. num은 지정 안 해놨어서 오류 출력


In [ ]:
# 명대사 가져온 결과 list 만드는 함수
def get_result(result):
  html_quotes = driver.page_source
  soup_quotes = bs4.BeautifulSoup(html_quotes, 'lxml')
  container = soup_quotes.select("#iframeDiv > ul > li")
  for quotes in container:
    quote = quotes.select_one("div > p.one_line").text
    result.append(quote)
  return result

In [ ]:
# 영화 결과 가져오기
def movie_result(movie_code, total):
  result = []
  if total > 10:
    # try:
    pages = total//10 # 명대사 한 쪽에 최대 10개. 총 대사 수를 10으로 나눠서 페이지 수 구하기
    pages_left = total%10 # 10으로 나눠떨어지지 않는 경우
    if pages_left != 0: # 그럴 땐 페이지 하나 추가  
      pages += 1
    for page in range (1, pages+1):
      page_url = f"https://movie.naver.com/movie/bi/mi/script.naver?code={movie_code}&order=best&nid=&page={page}"
      driver.get(page_url)
      #driver.switch_to.frame('ifr_area') # iframe name 필요 없음
      get_result(result)
    return result
  else:
    try: # 명대사가 있으면
      # iframe 형식으로 구성됨. 바꿔줘야함.
      quotes_url = f"https://movie.naver.com/movie/bi/mi/scriptAndRelate.naver?code={movie_code}"
      driver.get(quotes_url)
      driver.switch_to.frame('ifr_lines') # iframe name
      get_result(result)
      return result
    except: # 없으면
      return result # pass해서 None값 나오면 다음 함수에서 처리 안 됨

In [ ]:
from tqdm.auto import tqdm # for문 진행상황 보여주기
results = [] # 전체 결과
total_codes = [] # 전체 영화 코드

# 실행 확인 for movie in title_list[0:3]:
for movie in tqdm(title_list): 
  code_list = get_code_list(movie)
  total_codes+=code_list



  0%|          | 0/1012 [00:00<?, ?it/s]

In [ ]:
print(len(total_codes))
result = []
results = []
quote_table = []
result_table = []


2542
['207921', '17640', '191686', '154562', '81350', '154282', '77550', '103794', '52369', '149781']


In [ ]:
# Colab에서 Selenium 사용하기
!apt-get update
!apt install chromium-chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubu

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
for code in tqdm(total_codes[:10]):
  num = get_num(code)
  result = movie_result(code, num)
  results += result # 하나의 리스트로 만들기



  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
quote_table = pd.DataFrame(results, columns= ['quote'])
result_table=quote_table.drop_duplicates(ignore_index=True) # 중복제거, 인덱스 재설정

In [ ]:
print(result_table)

                                                quote
0           전하는 치료가 아니라 사랑이 필요해요. 사랑, 충격, 웃음. 아주 많이요.
1                    그레고리에게 전해주세요. 110kg이라고. 충분히 즐겼네요
2            IT'S NOT JUST ME WHO LOVES YOU!  ♡Maggie
3                                     아름다움은 껍데기일 뿐이에요
4                                   어차피 몸무게 절반은 장신구에요
5                    -왜 운전해서 오셨어요? -차는 저절로 움직이지 않으니까요
6                                          내일은 선거일이야.
7                                              나는 복수다
8                                      크르르르 못 참겠다 로빈!
9                 적들은 내가 그림자 속에 숨는 줄 알지만, 내가 바로 그림자다.
10                                  나 감독 안해요! 내가 안해요!
11                                        아가츄! 아.가.츄!
12                                  난 베트맨이아니라 배트맨이라구!
13                                          재개발은 거짓이야
14                                               안아줘요
15                        마리아~ 아베 마리아~ 저 흰 구름 끝까지 날아~
16          나는 잔인하고 시적이며 눈이 멀었다네 그런 나를 부정하고 폭력으로 규정하네
17                          

In [ ]:
result_table.to_csv('/content/drive/MyDrive/Quotes_and_Recommend/data/quote.csv')

### Data 양이 많아서 나눠서 저장

In [ ]:
result = []
results = []
quote_table = []
result_table = []

In [ ]:
for code in tqdm(total_codes[:100]):
  num = get_num(code)
  result = movie_result(code, num)
  results += result # 하나의 리스트로 만들기

quote_table = pd.DataFrame(results, columns= ['quote'])
result_table=quote_table.drop_duplicates(ignore_index=True) # 중복제거, 인덱스 재설정
result_table.to_csv('/content/drive/MyDrive/Quotes_and_Recommend/data/quote1.csv')

  0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
result = []
results = []
quote_table = []
result_table = []
for code in tqdm(total_codes[100:300]):
  num = get_num(code)
  result = movie_result(code, num)
  results += result # 하나의 리스트로 만들기

quote_table = pd.DataFrame(results, columns= ['quote'])
result_table=quote_table.drop_duplicates(ignore_index=True) # 중복제거, 인덱스 재설정
result_table.to_csv('/content/drive/MyDrive/Quotes_and_Recommend/data/quote2.csv')

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
result = []
results = []
quote_table = []
result_table = []
for code in tqdm(total_codes[300:1000]):
  num = get_num(code)
  result = movie_result(code, num)
  results += result # 하나의 리스트로 만들기

quote_table = pd.DataFrame(results, columns= ['quote'])
result_table=quote_table.drop_duplicates(ignore_index=True) # 중복제거, 인덱스 재설정
result_table.to_csv('/content/drive/MyDrive/Quotes_and_Recommend/data/quote3.csv')

  0%|          | 0/700 [00:00<?, ?it/s]